**找到常微分方程的解**
考虑简单的常微分方程

$$ \frac {dy}{dx} =1 ,y(0)=1 $$


In [17]:
#利用sympy来解ode
from sympy import symbols, Function,solve
import numpy as np
import matplotlib.pyplot as plt

In [18]:
from sympy import Function, dsolve, Derivative, symbols,lambdify

t = symbols('t')
y = Function('y')(t)

# 定义微分方程
ode = Derivative(y, t) - 1
# 初始条件
ics = {y.subs(t, 0): 0}
# 求解微分方程
sol = dsolve(ode,ics=ics)

print(sol)
# 将解析解转为可用于数值计算的函数
f = lambdify(t, sol.rhs, "numpy")

# 创建 x 的值
t_vals = np.linspace(0, 999, 1000)

# 计算对应的 y 的值
y_vals = f(t_vals)

print(y_vals)

Eq(y(t), t)
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194.

**先从ode入手**
这里我们构造一些数据集
为了简化，变量如下 $$ t \in [0,10],dt=0.1$$ 暂时只涉及基础的ode，一阶导 y‘=c和y’=a+bt这样的常微分方程
构造的数据在一个batch是这样的：$$[ y_0(initial condition),t(variable)，y \prime(information of derivatation),y]$$
举个例子：$$[1,2,1,y] 表示 y_0=1,t=2时候，y\prime=1,label是y$$
$$[1,2,[1,1],y] 表示 y_0=1,t=2时候，y\prime=1+1*t，label是y$$
在这次的研究中，$$ 训练集的initial condition =0 $$
$$ c\in [1,10]，间隔1$$
$$ a\in [1,10],b\in [1,10],间隔0.1$$
$$ 测试集的initial condition =0 $$
$$ c\in [1,20]，间隔1$$
$$ a\in [1,10],b\in [1,10],间隔0.1$$


In [ ]:
#写自动生成数据集
import numpy as np
from sympy import Function, dsolve, Derivative, symbols,lambdify
import matplotlib.pyplot as plt
t_length=100
c_min=1
c_max=11
def prodeuce_ode_data(c_min=1,c_max=11):
    t = symbols('t')
    y = Function('y')(t)
    ics = {y.subs(t, 0.0): 0.0}
    t_vals = np.linspace(0, 10, t_length)
    t_vals=t_vals.reshape(t_length,1)
    print("数据集")
    whole_data=np.zeros([1,100,4])
    print(whole_data.shape)
    for c in range(c_min,c_max):
        ode = Derivative(y, t) - c
        sol = dsolve(ode, ics=ics)
        f = lambdify(t, sol.rhs, "numpy")
        y_vals = f(t_vals)  #
        y_vals=y_vals.reshape(t_length,1)   #y
        y_prime=np.ones((t_length,1))*c #y_prime
        y_initial=np.zeros((t_length,1)) #initial condition
        data=np.concatenate((y_initial,t_vals,y_prime,y_vals),axis=1)
        whole_data=np.concatenate((whole_data,data.reshape(1,100,4)),axis=0)
    whole_data=whole_data[1:,:,:]
    return whole_data
train_data=prodeuce_ode_data(c_min=1,c_max=11)
test_data=prodeuce_ode_data(c_min=10,c_max=20)
print(train_data.shape)

plt.style.use('ggplot')
# 画出图像 一行两张图
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i in range (c_min-1,c_max-1):

    axes[0].plot(train_data[i,:,1],train_data[i,:,3],label=f'train_data {i+1}')
    axes[1].plot(test_data[i,:,1],test_data[i,:,3],label=f'test_data {i+1}')
    #两张图的横纵坐标和legend
    axes[0].set_xlabel("t")
    axes[0].set_ylabel("y")
    axes[1].set_xlabel("t")
    axes[1].set_ylabel("y")
    axes[0].legend()
    axes[1].legend()


数据集生成完毕
保存为tensor


In [70]:
import torch
train_data_tensor=torch.from_numpy(train_data)
test_data_tensor=torch.from_numpy(test_data)
#保存到.pt文件
torch.save(train_data_tensor,'./ode_dataset/train_data.pt')
torch.save(test_data_tensor,'./ode_dataset/test_data.pt')
print("save done")


save done
